In [1]:
import numpy as np
import pandas as pd

In [2]:
from typing import List, Dict

In [3]:
!pip install scikit-learn

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [25]:
songs = pd.read_csv(r'C:\Users\HP\OneDrive\Desktop\Music Recommendation\songdata.csv')

In [26]:
songs.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [27]:
songs = songs.sample(n=5000).drop('link', axis=1).reset_index(drop=True)

In [29]:
print(songs)

               artist                     song  \
0           Kim Wilde   Say You Really Want Me   
1               Ne-Yo            It's All Good   
2           Green Day           Minnesota Girl   
3           Zebrahead     Playmate Of The Year   
4         Carly Simon           Devoted To You   
...               ...                      ...   
4995            Clash        Red Angel Dragnet   
4996    Conway Twitty              All My Life   
4997              HIM        Gone With The Sin   
4998     Louis Jordan  Let The Good Times Roll   
4999  Nine Inch Nails                   Ruiner   

                                                   text  
0     Why you always got to be so cool  \nAnd why I ...  
1     Ne-Yo:  \nOK! Bad day got you seeing red  \nWa...  
2     Sittin' on the hood of a camaro  \nWalkin' sho...  
3     There's this girl I don't know  \nComes by eve...  
4     Darlin' you can count on me  \nTill the sun dr...  
...                                                

In [30]:
songs['text'] = songs['text'].str.replace(r'\n', '')

In [31]:
print(songs)

               artist                     song  \
0           Kim Wilde   Say You Really Want Me   
1               Ne-Yo            It's All Good   
2           Green Day           Minnesota Girl   
3           Zebrahead     Playmate Of The Year   
4         Carly Simon           Devoted To You   
...               ...                      ...   
4995            Clash        Red Angel Dragnet   
4996    Conway Twitty              All My Life   
4997              HIM        Gone With The Sin   
4998     Louis Jordan  Let The Good Times Roll   
4999  Nine Inch Nails                   Ruiner   

                                                   text  
0     Why you always got to be so cool  \nAnd why I ...  
1     Ne-Yo:  \nOK! Bad day got you seeing red  \nWa...  
2     Sittin' on the hood of a camaro  \nWalkin' sho...  
3     There's this girl I don't know  \nComes by eve...  
4     Darlin' you can count on me  \nTill the sun dr...  
...                                                

In [32]:
songs['text'] = songs['text'].str.replace(r'\n', '')

In [34]:
tfidf = TfidfVectorizer(analyzer='word', stop_words='english')

In [35]:
lyrics_matrix = tfidf.fit_transform(songs['text'])

In [36]:
cosine_similarities = cosine_similarity(lyrics_matrix) 

In [37]:
similarities = {}

In [38]:
for i in range(len(cosine_similarities)):
    # Now we'll sort each element in cosine_similarities and get the indexes of the songs. 
    similar_indices = cosine_similarities[i].argsort()[:-50:-1] 
    # After that, we'll store in similarities each name of the 50 most similar songs.
    # Except the first one that is the same song.
    similarities[songs['song'].iloc[i]] = [(cosine_similarities[i][x], songs['song'][x], songs['artist'][x]) for x in similar_indices][1:]

In [39]:
class ContentBasedRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix

    def _print_message(self, song, recom_song):
        rec_items = len(recom_song)
        
        print(f'The {rec_items} recommended songs for {song} are:')
        for i in range(rec_items):
            print(f"Number {i+1}:")
            print(f"{recom_song[i][1]} by {recom_song[i][2]} with {round(recom_song[i][0], 3)} similarity score") 
            print("--------------------")
        
    def recommend(self, recommendation):
        # Get song to find recommendations for
        song = recommendation['song']
        # Get number of songs to recommend
        number_songs = recommendation['number_songs']
        # Get the number of songs most similars from matrix similarities
        recom_song = self.matrix_similar[song][:number_songs]
        # print each item
        self._print_message(song=song, recom_song=recom_song)

In [41]:
recommedations = ContentBasedRecommender(similarities)

In [42]:
recommendation = {
    "song": songs['song'].iloc[10],
    "number_songs": 4 
}

In [43]:
recommedations.recommend(recommendation)

The 4 recommended songs for Ribbons Of Blue are:
Number 1:
How Do I Wrap My Heart Up For Christmas by Christmas Songs with 0.278 similarity score
--------------------
Number 2:
Blue Me by Dolly Parton with 0.262 similarity score
--------------------
Number 3:
How Do I Wrap My Heart Up For Christmas by Randy Travis with 0.257 similarity score
--------------------
Number 4:
Blue Christmas by Harry Connick, Jr. with 0.233 similarity score
--------------------


In [44]:

recommendation2 = {
    "song": songs['song'].iloc[120],
    "number_songs": 4 
}

In [45]:

recommedations.recommend(recommendation2)

The 4 recommended songs for Modern Eyes are:
Number 1:
I Can't Take My Eyes Off You by Josh Groban with 0.234 similarity score
--------------------
Number 2:
Modern Day Delilah by Kiss with 0.227 similarity score
--------------------
Number 3:
A Modern Day Version Of Love by Neil Diamond with 0.209 similarity score
--------------------
Number 4:
She Closed Her Eyes by Chris Rea with 0.19 similarity score
--------------------
